In [19]:
import torch.nn as nn
class SMSClassifierModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__();
        self.model_input = nn.Embedding(vocab_size, 100);
        # self.rnn = nn.RNN(100, 128);
        self.rnn = nn.LSTM(100, 128);
        self.model_output = nn.Linear(128, 1);
        self.activation = nn.Sigmoid()
        
    def forward(self, sms_text):
        embedded = self.model_input(sms_text);
        # _,hidden = self.rnn(embedded);
        _,(hidden,_) = self.rnn(embedded);
        output = self.model_output(hidden);
        return self.activation(output)

In [20]:
import torch
import os.path
import pickle

def read_vocab(path):
    pkl_file = open(path, 'rb')
    vocab = pickle.load(pkl_file)
    pkl_file.close()
    return vocab

def load_model_and_vocab(vocab_path, weights_path):    
    vocab = read_vocab(vocab_path)
    model = SMSClassifierModel(vocab_size=len(vocab))
    model.load_state_dict(torch.load(weights_path))
    model.eval()
    return model, vocab

model, vocab = load_model_and_vocab("sms_vocab.pkl", "sms_classifier_weights.pt");

In [21]:
import nltk
nltk.download("punkt");
from nltk import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
from torch.nn.utils.rnn import pad_sequence
import pandas as pd

data = pd.read_csv("../datasets/sms-classification/spam_test.csv", encoding="latin-1");
data.head()
indexed = []
for index, row in data.iterrows():    
    tokenized = word_tokenize(row['text']);
    index_sentence = torch.LongTensor([vocab.stoi[t] for t in tokenized]);
    indexed.append(index_sentence);
    
indexed_padded = pad_sequence(indexed, batch_first=True, padding_value=1)
input_tensor = torch.LongTensor(indexed_padded).transpose(0,1);
print(input_tensor.shape); # [sentence_length, batch_size]
pred = model(input_tensor).squeeze();
correct = 0
n = 0
for i,p in enumerate(pred):
    p = torch.round(p);
    label = {0: "ham", 1:"spam"};
    label_rev = {"ham":0, "spam":1};
    correct += p.item()==label_rev[data.iloc[i]['label']]
#     if not p.item()==label_rev[data.iloc[i]['label']]:
#         print(n, data.iloc[i]['label'])
#         n+=1
    print(data.iloc[i]['text'], " => ", label[p.item()], " ", data.iloc[i]['label']);
print(correct*100/len(data.index),"%");

torch.Size([196, 1115])
Funny fact Nobody teaches volcanoes 2 erupt, tsunamis 2 arise, hurricanes 2 sway aroundn no 1 teaches hw 2 choose a wife Natural disasters just happens  =>  spam   ham
I sent my scores to sophas and i had to do secondary application for a few schools. I think if you are thinking of applying, do a research on cost also. Contact joke ogunrinde, her school is one me the less expensive ones  =>  ham   ham
We know someone who you know that fancies you. Call 09058097218 to find out who. POBox 6, LS15HB 150p  =>  ham   spam
Only if you promise your getting out as SOON as you can. And you'll text me in the morning to let me know you made it in ok.  =>  ham   ham
Congratulations ur awarded either Ã¥Â£500 of CD gift vouchers & Free entry 2 our Ã¥Â£100 weekly draw txt MUSIC to 87066 TnCs www.Ldew.com1win150ppmx3age16  =>  spam   spam
I'll text carlos and let you know, hang on  =>  ham   ham
K.i did't see you.:)k:)where are you now?  =>  ham   ham
No message..no responce..w

Edison has rightly said, \A fool can ask more questions than a wise man can answer\" Now you know why all of us are speechless during ViVa.. GM  =>  ham   ham
How many licks does it take to get to the center of a tootsie pop?  =>  ham   ham
Hello, my love. What are you doing? Did you get to that interview today? Are you you happy? Are you being a good boy? Do you think of me?Are you missing me ?  =>  ham   ham
As a valued customer, I am pleased to advise you that following recent review of your Mob No. you are awarded with a Ã¥Â£1500 Bonus Prize, call 09066364589  =>  ham   spam
How's it going? Got any exciting karaoke type activities planned? I'm debating whether to play football this eve. Feeling lazy though.  =>  ham   ham
Too late. I said i have the website. I didn't i have or dont have the slippers  =>  ham   ham
Yetunde i'm in class can you not run water on it to make it ok. Pls now.  =>  ham   ham
Here is your discount code RP176781. To stop further messages reply stop. www.rega

And by when you're done I mean now  =>  ham   ham
What you doing?how are you?  =>  ham   ham
Bloomberg -Message center +447797706009 Why wait? Apply for your future http://careers. bloomberg.com  =>  spam   spam
Did u got that persons story  =>  ham   ham
Nah can't help you there, I've never had an iphone  =>  ham   ham
Sorry, I'll call later  =>  ham   ham
2/2 146tf150p  =>  ham   spam
A boy was late 2 home. His father: \POWER OF FRNDSHIP\""  =>  ham   ham
How are you. Just checking up on you  =>  ham   ham
I hope you that's the result of being consistently intelligent and kind. Start asking him about practicum links and keep your ears open and all the best. ttyl  =>  ham   ham
\Hi darlin i cantdo anythingtomorrow as myparents aretaking me outfor a meal. when are u free? Katexxx\""  =>  ham   ham
Hey you can pay. With salary de. Only  &lt;#&gt; .  =>  ham   ham
Yeah do! DonÂÃÃ·t stand to close tho- youÂÃÃ·ll catch something!  =>  ham   ham
Ok i msg u b4 i leave my house.  =>  ham 